# Importing Libraries, Mounting drive, importing dataset and fixing up dataframe

In [ ]:
!pip install transformers
!pip install rouge-score==0.0.4
import pandas as pd
import numpy as np
from transformers import pipeline
from google.colab import drive
from rouge_score import rouge_scorer
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 18.0 MB/s 
     |████████████████████████████████| 596 kB 26.2 MB/s 
     |████████████████████████████████| 6.6 MB 23.9 MB/s 
     |████████████████████████████████| 101 kB 6.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/practiCUM/summerizer/BookList_tiny.csv', delimiter =  "~")
df.drop(columns = 'Unnamed: 0', inplace = True)


In [ ]:
df.rename(columns = {'bart_large_xsum_samsum':'google/pegasus-xsum'}, inplace = True)
df.drop(columns = 'full_text')
drops = ['full_text','rouge1','rouge2','rougeL','bart_large_xsum_samsum\t','book_wordcount','sum_wordcount','gensum_wordcount']
df = df.drop(columns = drops)
df

,title,summary,google/pegasus-xsum
0,The Call of the Wild,"The story opens in 1897 with Buck, a powerful ...",In our series of letters from African-American...
1,The Wonderful Wizard of Oz,Dorothy is a young girl who lives with her Aun...,All photographs by Geo M. Hill Co. All images ...
2,Much Ado about Nothing,"In Messina, a messenger brings news that Don P...",In our series of letters from Shakespeare's cr...
3,Candide,"Candide contains thirty episodic chapters, whi...",The complete list of titles in the Modern Libr...
4,The Hound of the Baskervilles,Dr James Mortimer calls on Sherlock Holmes in ...,The Hound of the Baskervilles by Sir Arthur Co...
5,Crome Yellow,A house party at Crome is viewed largely throu...,"Denis sat down, took a deep breath, and began ..."
6,Lady into Fox,"Silvia Tebrick, the 24-year-old wife of Richar...",In our series of letters from African-American...
7,The Great Gatsby,"In spring 1922, Nick Carraway—a Yale alumnus f...",In our series of letters from African-American...
8,The Island of Doctor Moreau,Edward Prendick is an Englishman with a scient...,The following is an extract from the letter of...
9,The Mysterious Affair at Styles,"On the morning of 18 July, the household at St...",The following is the full text of Agatha Chris...


# Creating and applying a ROUGE function

In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)

In [ ]:
df["rouge"]= df.apply(lambda x: scorer.score(x['google/pegasus-xsum'], x['summary']), axis=1)



In [ ]:
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
#df.head(106)
#df["rouge"]
def unpack_rouge(column_name):
  df[column_name+"1"] = df[column_name].map(lambda x: x["rouge1"].fmeasure) 
  df[column_name+"2"] = df[column_name].map(lambda x: x["rouge2"].fmeasure) 
  df[column_name+"L"] = df[column_name].map(lambda x: x["rougeL"].fmeasure) 

In [ ]:
unpack_rouge("rouge")

# Results

In [ ]:
df.describe()

,rouge1,rouge2,rougeL
count,10.000000,10.000000,10.000000
mean,0.300761,0.075160,0.138222
std,0.132025,0.046153,0.049250
min,0.100231,0.015444,0.074017
25%,0.179450,0.037715,0.096220
50%,0.320333,0.072847,0.139592
75%,0.397211,0.101732,0.165787
max,0.470141,0.167491,0.220616


In [ ]:
drop =['rouge']
df=df.drop(columns = drop)


# Save and extract as csv

In [ ]:
from google.colab import drive

path = '/content/drive/MyDrive/practiCUM/summerizer/rouge_small_results_4_pegasus.csv'

with open(path, 'w') as f:
  df.to_csv(f)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/practiCUM/summerizer/rouge_small_results_4_pegasus.csv', delimiter =  ",")
df.drop(columns = 'Unnamed: 0', inplace = True)

In [ ]:
pd.set_option('display.max_rows', None)
df.head(100)

,title,summary,google/pegasus-xsum,rouge1,rouge2,rougeL
0,The Call of the Wild,"The story opens in 1897 with Buck, a powerful ...",In our series of letters from African-American...,0.453139,0.102004,0.190173
1,The Wonderful Wizard of Oz,Dorothy is a young girl who lives with her Aun...,All photographs by Geo M. Hill Co. All images ...,0.470141,0.167491,0.220616
2,Much Ado about Nothing,"In Messina, a messenger brings news that Don P...",In our series of letters from Shakespeare's cr...,0.320479,0.046605,0.140957
3,Candide,"Candide contains thirty episodic chapters, whi...",The complete list of titles in the Modern Libr...,0.100231,0.015444,0.074017
4,The Hound of the Baskervilles,Dr James Mortimer calls on Sherlock Holmes in ...,The Hound of the Baskervilles by Sir Arthur Co...,0.372626,0.100917,0.160445
5,Crome Yellow,A house party at Crome is viewed largely throu...,"Denis sat down, took a deep breath, and began ...",0.145951,0.032045,0.075330
6,Lady into Fox,"Silvia Tebrick, the 24-year-old wife of Richar...",In our series of letters from African-American...,0.149176,0.034752,0.084996
7,The Great Gatsby,"In spring 1922, Nick Carraway—a Yale alumnus f...",In our series of letters from African-American...,0.320187,0.086753,0.138227
8,The Island of Doctor Moreau,Edward Prendick is an Englishman with a scient...,The following is an extract from the letter of...,0.405405,0.106646,0.167568
9,The Mysterious Affair at Styles,"On the morning of 18 July, the household at St...",The following is the full text of Agatha Chris...,0.270270,0.058942,0.129891
